# Metagenome Reconstruction

Assembly of MAGS from pool of all metagenomic fastq files. 

The fastq files will need to be placed in a `fastq` subdirectory of this directory, named `sampleid_1.fq.gz`, `sampleid_2.fq.gz` etc.

The 'heavy lifting' is handled by the compute cluster - ensure a set of jobs from a cell running a `qsub` command have completed successfully prior to running the next cell.

The contents of the conda environment used to run this analysis if contained within the `mag_assembly.conda.yaml` file, which should allow the environment to be recreated using `conda env create -f mag_assembly.conda.yaml`

In [6]:
import pandas as pd
import numpy as np
import glob
import os
from Bio import SeqIO
import re
from natsort import index_natsorted
from IPython.display import display, Markdown,HTML, IFrame

pd.set_option('display.max_rows',70)

## Assembly

Assembly of all samples and replicates is handled together, using megahit which handles memory in a friendlier way than metaspades.

In [ ]:
!qsub bin/megahit.sh

Output from assembly-stats:

| filename	| total_length	| number	| mean_length |	longest	| shortest	| N_count |	Gaps	| N50	| N50n	| N70	| N70n	| N90	| N90n |
|--|--|--|--|--|--|--|--|--|--|--|--|--|--|
| megahit/final.contigs.fa |	12841009562	| 21005959	| 611.30	| 579848	| 200	| 0 |	0	| 627	| 5320588	| 450	| 10208120	| 339	| 16810361 |

## Read alignment

metabat2 requires a file containing mean and variance of base coverage for each contig, which can be derived from bam files using the metabat `jgi_summarize_bam_contig_depths` script. First we need bam files for each sample/replicate. First...index the assemblies:

In [ ]:
!qsub bin/bwa_index.sh -r megahit/final.contigs.fa

Now align each samples reads against the assemblies...The bwa indexes were copied to the `reference` directory between running these commands...

In [ ]:
!qsub -t 1-12 bin/align_reference.sh -f fastq -o bwa_alignments -d reference -r final.contigs.fa

## Read Depth Summary

The `jgi_summarize_bam_contigs` script is used to produce coverage statistics for each contig in each sample

In [ ]:
!qsub bin/summarise_depth.sh

## Metabat

Binning is carried out using metabat2, which self-tunes parameters. As a first pass just use defaults for everything else and see what happens...

In [ ]:
!qsub bin/metabat.sh

Now use checkm to see how the results stack up...

In [ ]:
!qsub bin/checkm.sh

## Dereplication

Replicates are removed using dRep...

In [ ]:
!qsub bin/drep.sh

checkm is then again used to assess the resulting genomes...

In [146]:
!cat bin/checkm2.sh

#!/usr/bin/env bash

#$ -cwd
#$ -j y
#$ -mods l_hard mfree 64G
#$ -pe smp 24
#$ -o job_logs/$JOB_NAME.$JOB_ID

checkm tree -t 24 -x fa drep/dereplicated_genomes drep_checkm
checkm tree_qa -t 24 drep_checkm
checkm lineage_set drep_checkm drep_checkm/markers
checkm analyze -t 24 -x fa drep_checkm/markers drep/dereplicated_genomes drep_checkm
checkm qa -t 24 -o 1 --tab_table -f drep_checkm/results.tsv drep_checkm/markers drep_checkm



The `plot_checkm.R` notebook was used to create a plot of the results...

![checkm_plot](checkm.png)

## Taxonomy assignment

Taxonomy assignment was carried out using gtdb-tk's classify_wf workflow.

In [ ]:
!qsub bin/gtdbtk.sh

Parsing the GTDB results into a more readable format...

In [11]:
with open('gtdbtk/gtdbtk.bac120.summary.tsv','r') as fh:
        head=fh.readline() #ditch header
        lines=fh.readlines()
fh.close()

outfh=open('gtdbtk/parsed_taxonomy.txt','w')

outfh.write("Bin\tKingdom\tPhylum\tClass\tOrder\tFamily\tGenus\tSpecies\n")
for line in lines:
        out_classes=list()
        elems=line.split('\t')
        bin=elems[0]
        classification=elems[1]

        classes=classification.split(';')
        for c in classes:
                c=re.sub(r'[a-z]__','',c)
                if c=="":
                        c="unclassified"
                out_classes.append(c)

        outline="\t".join(out_classes)
        outfh.write("{}\t{}\n".format(bin,outline))
outfh.close()

gtdbtk_res=pd.read_csv('gtdbtk/parsed_taxonomy.txt',sep="\t")
gtdbtk_res=gtdbtk_res.sort_values(by='Bin', key=lambda x: np.argsort(index_natsorted(gtdbtk_res["Bin"])))
display(gtdbtk_res)

Bin   Kingdom             Phylum                Class  \
54    bin.5  Bacteria     Proteobacteria  Alphaproteobacteria   
1    bin.11  Bacteria    Gemmatimonadota     Gemmatimonadetes   
17   bin.19  Bacteria       Bacteroidota          Bacteroidia   
20   bin.21  Bacteria       Bacteroidota          Bacteroidia   
21   bin.22  Bacteria       Bacteroidota          Bacteroidia   
29   bin.24  Bacteria       Bacteroidota          Bacteroidia   
50   bin.37  Bacteria    Gemmatimonadota     Gemmatimonadetes   
51   bin.42  Bacteria     Proteobacteria  Alphaproteobacteria   
52   bin.48  Bacteria     Proteobacteria  Alphaproteobacteria   
53   bin.49  Bacteria       Bacteroidota          Bacteroidia   
55   bin.52  Bacteria       Bacteroidota          Bacteroidia   
56   bin.53  Bacteria     Proteobacteria  Gammaproteobacteria   
57   bin.58  Bacteria  Methylomirabilota     Methylomirabilia   
58   bin.61  Bacteria     Proteobacteria  Alphaproteobacteria   
59   bin.63  Bacteria     Proteobacteria  Gammaproteobacteria   
60   bin.74  Bacteria       Bacteroidota          Bacteroidia   
61   bin.84  Bacteria     Proteobacteria  Gammaproteobacteria   
62   bin.90  Bacteria     Proteobacteria  Gammaproteobacteria   
63   bin.96  Bacteria     Proteobacteria  Alphaproteobacteria   
64   bin.97  Bacteria    Gemmatimonadota     Gemmatimonadetes   
65   bin.98  Bacteria     Proteobacteria  Alphaproteobacteria   
66   bin.99  Bacteria   Actinobacteriota        Actinomycetia   
0   bin.109  Bacteria       Zixibacteria              MSB-5A5   
2   bin.111  Bacteria     Proteobacteria  Alphaproteobacteria   
3   bin.119  Bacteria       Bacteroidota          Bacteroidia   
4   bin.122  Bacteria       Bacteroidota          Bacteroidia   
5   bin.126  Bacteria       Bacteroidota          Bacteroidia   
6   bin.128  Bacteria        Myxococcota            Polyangia   
7   bin.142  Bacteria       Bacteroidota          Bacteroidia   
8   bin.151  Bacteria     Proteobacteria  Gammaproteobacteria   
9   bin.152  Bacteria       Bacteroidota          Bacteroidia   
10  bin.158  Bacteria       Bacteroidota          Bacteroidia   
11  bin.165  Bacteria     Proteobacteria  Gammaproteobacteria   
12  bin.169  Bacteria       Bacteroidota          Bacteroidia   
13  bin.174  Bacteria       Bacteroidota          Bacteroidia   
14  bin.175  Bacteria     Proteobacteria  Gammaproteobacteria   
15  bin.178  Bacteria       Bacteroidota          Bacteroidia   
16  bin.180  Bacteria   Bdellovibrionota      Bdellovibrionia   
18  bin.200  Bacteria     Proteobacteria  Gammaproteobacteria   
19  bin.208  Bacteria       Bacteroidota          Bacteroidia   
22  bin.221  Bacteria    Elusimicrobiota        Elusimicrobia   
23  bin.222  Bacteria     Proteobacteria  Gammaproteobacteria   
24  bin.223  Bacteria     Proteobacteria  Gammaproteobacteria   
25  bin.224  Bacteria       Bacteroidota          Bacteroidia   
26  bin.227  Bacteria   Bdellovibrionota      Bacteriovoracia   
27  bin.231  Bacteria       Bacteroidota          Bacteroidia   
28  bin.238  Bacteria       Bacteroidota          Bacteroidia   
30  bin.242  Bacteria     Armatimonadota       Fimbriimonadia   
31  bin.252  Bacteria   Actinobacteriota      Thermoleophilia   
32  bin.259  Bacteria        Myxococcota            Polyangia   
33  bin.268  Bacteria       Bacteroidota          Bacteroidia   
34  bin.272  Bacteria       Bacteroidota          Bacteroidia   
35  bin.273  Bacteria     Proteobacteria  Gammaproteobacteria   
36  bin.297  Bacteria     Proteobacteria  Alphaproteobacteria   
37  bin.303  Bacteria     Proteobacteria  Gammaproteobacteria   
38  bin.310  Bacteria     Proteobacteria  Alphaproteobacteria   
39  bin.312  Bacteria     Proteobacteria  Alphaproteobacteria   
40  bin.319  Bacteria       Bacteroidota          Bacteroidia   
41  bin.324  Bacteria       Bacteroidota          Bacteroidia   
42  bin.328  Bacteria       Bacteroidota          Bacteroidia   
43  bin.329  Bacteria     Proteobacteria  G

Now pull together a summary table of checkm and gtdb-tk results...

In [14]:
checkm=pd.read_csv('drep_checkm/results.tsv',sep='\t')
checkm=checkm[['Bin Id','Marker lineage','Completeness','Contamination','Strain heterogeneity']]

gtdbtk=pd.read_csv('gtdbtk/gtdbtk.bac120.summary.tsv',sep='\t')
gtdbtk=gtdbtk[['user_genome','classification']]

summary=checkm.merge(gtdbtk,how='inner',left_on='Bin Id',right_on='user_genome')
summary=summary[['Bin Id','Marker lineage','Completeness', 'Contamination','Strain heterogeneity']]
summary.to_csv('MAG_summary.txt',sep='\t',index=False)

Next let's try to get rid of some  contamination with magpurify...

In [ ]:
%%bash

mkdir magpurify
qsub bin/magpurify.sh
qsub bin/checkm3.sh

In [15]:
magpurify=pd.read_csv('magpurify_checkm/results.tsv',sep='\t')
magpurify=magpurify[['Bin Id','Completeness','Contamination','Strain heterogeneity']]

summary=summary.merge(magpurify,how='left',left_on='Bin Id',right_on='Bin Id')
summary.rename(columns={'Completeness_x':'pre-purify completeness',
                        'Contamination_x':'pre-purify contamination',
                        'Strain heterogeneity_x':'pre-purify heterogeneity',
                        'Completeness_y':'post-purify completeness',
                        'Contamination_y':'post-purify contamination',
                        'Strain heterogeneity_y':'post-purify heterogeneity'},inplace=True)

In [16]:
fastas=glob.glob('magpurify/*.fa')
assemb_stats=pd.DataFrame(columns=('id','contigs','length'))
for fasta in fastas:
    bin=os.path.basename(fasta).replace('.fa','')
    assem_length=0
    contigs=0
    with open(fasta,'r') as fh:
        for record in SeqIO.parse(fh,'fasta'):
            assem_length=assem_length+len(record.seq)
            contigs+=1
    assemb_stats=assemb_stats.append({'id':bin,'contigs':contigs,'length':assem_length},ignore_index=True)
summary=summary.merge(assemb_stats,how='left',left_on='Bin Id', right_on='id')
summary=summary.drop(columns=['id'])
summary=summary.sort_values(by='Bin Id', key=lambda x: np.argsort(index_natsorted(summary["Bin Id"])))
display(summary)

Bin Id                    Marker lineage  pre-purify completeness  \
54    bin.5     o__Sphingomonadales (UID3310)                    98.96   
1    bin.11             k__Bacteria (UID2495)                    87.91   
17   bin.19             k__Bacteria (UID2569)                    95.01   
20   bin.21        p__Bacteroidetes (UID2605)                    97.62   
21   bin.22              k__Bacteria (UID203)                    75.86   
29   bin.24        p__Bacteroidetes (UID2591)                    88.55   
50   bin.37             k__Bacteria (UID2495)                    81.39   
51   bin.42          o__Rhizobiales (UID3447)                    80.09   
52   bin.48     o__Sphingomonadales (UID3310)                    83.32   
53   bin.49         o__Cytophagales (UID2936)                    99.09   
55   bin.52              k__Bacteria (UID203)                    80.50   
56   bin.53              k__Bacteria (UID203)                    75.17   
57   bin.58             k__Bacteria (UID3187)                    79.56   
58   bin.61     o__Sphingomonadales (UID3310)                    92.77   
59   bin.63      o__Burkholderiales (UID4000)                    91.40   
60   bin.74        p__Bacteroidetes (UID2605)                    97.07   
61   bin.84     f__Xanthomonadaceae (UID4214)                    96.01   
62   bin.90      o__Burkholderiales (UID4000)                    88.88   
63   bin.96          o__Rhizobiales (UID3450)                    95.65   
64   bin.97             k__Bacteria (UID2495)                    95.34   
65   bin.98  c__Alphaproteobacteria (UID3422)                    98.18   
66   bin.99       f__Micrococcaceae (UID1631)                    81.98   
0   bin.109             k__Bacteria (UID2495)                    96.89   
2   bin.111          o__Rhizobiales (UID3447)                    83.45   
3   bin.119             k__Bacteria (UID2569)                    96.99   
4   bin.122        p__Bacteroidetes (UID2605)                    92.48   
5   bin.126        p__Bacteroidetes (UID2605)                    78.16   
6   bin.128  c__Deltaproteobacteria (UID3216)                    83.08   
7   bin.142        p__Bacteroidetes (UID2591)                    91.70   
8   bin.151              k__Bacteria (UID203)                    95.92   
9   bin.152        p__Bacteroidetes (UID2591)                    86.99   
10  bin.158        p__Bacteroidetes (UID2591)                    92.22   
11  bin.165      o__Burkholderiales (UID4002)                    94.55   
12  bin.169        p__Bacteroidetes (UID2605)                    95.03   
13  bin.174        p__Bacteroidetes (UID2605)                    87.31   
14  bin.175   f__Enterobacteriaceae (UID5054)                    80.33   
15  bin.178        p__Bacteroidetes (UID2591)                    99.01   
16  bin.180             k__Bacteria (UID3187)                    76.16   
18  bin.200      o__Burkholderiales (UID4002)                    94.01   
19  bin.208        p__Bacteroidetes (UID2591)                    95.81   
22  bin.221             k__Bacteria (UID2495)                    77.06   
23  bin.222     f__Xanthomonadaceae (UID4214)                    87.77   
24  bin.223   c__Betaproteobacteria (UID3959)                    80.52   
25  bin.224         o__Cytophagales (UID2936)                    97.02   
26  bin.227             k__Bacteria (UID3187)                    90.85   
27  bin.231        p__Bacteroidetes (UID2605)                    98.89   
28  bin.238        p__Bacteroidetes (UID2605)                    90.36   
30  bin.242             k__Bacteria (UID1452)                    94.14   
31  bin.252       p__Actinobacteria (UID1454)                    91.73   
32  bin.259              k__Bacteria (UID203)                    78.50   
33  bin.268    f__Flavobacteriaceae (UID2817)                    98.85   
34  bin.272        p__Bacteroidetes (UID2605)                    99.05   
35  bin.273     f__Pseudomonadaceae (UID4490)                    93.19   
36  bin.2

MIMAG specifications for mags are as follows:
* high quality draft: completeness > 90% and contamination <5%; 23S, 16S and 5S rRNA and >18 tRNA
* medium quality draft: completeness > 50% and contamination <10%
* low quality draft: contamination <10%

Prokka was therefore run to identify rRNA and tRNA sequences to aid in classifying the MAGs:

In [152]:
!qsub bin/prokka

In [152]:
%%bash
cd prokka
echo -e "id\ttRNA" > tRNA_counts
echo -e "id\trRNA" > rRNA_counts
find . -name *txt -exec grep -H tRNA {} \;|tr " " "\t"|awk -F'/' '{print $2,"\t",$3}'|cut -f1,3|sed 's/ //' >> tRNA_counts
find . -name *tsv -exec grep -HEc '(16|23|5)S ribosomal RNA' {} \;|tr ":" "\t"|awk -F'/' '{print $2,"\t",$3}'|cut -f1,3 |sed 's/ //' >> rRNA_counts

In [17]:
RNA=pd.read_csv('prokka/tRNA_counts',sep='\t')
rRNA=pd.read_csv('prokka/rRNA_counts',sep='\t')
RNA=RNA.merge(rRNA,how='left',left_on='id',right_on='id')

summary=summary.merge(RNA,how='left',left_on='Bin Id',right_on='id')
summary.drop('id',inplace=True,axis=1)

summary=summary[["Bin Id","Marker lineage", "post-purify completeness", 
                 "post-purify contamination", "post-purify heterogeneity",
               "contigs","length","tRNA","rRNA"]]

summary.rename(columns={'post-purify completeness':'completeness', 
                        'post-purify contamination':'contamination', 
                        'post-purify heterogeneity':'heterogeneity'},
               inplace=True)
  

Now filter the MAGs according to completeness, contamination and t/rRNA counts

In [18]:
low_qual=summary[(summary['completeness']<50) & (summary['contamination']<10)]
med_qual=summary[(summary['completeness']>=50) & (summary['completeness']<90) & (summary['contamination']<10)]
high_contam=summary[(summary['completeness']>=90) & (summary['contamination']>5) & (summary['contamination']<10)]
med_qual=med_qual.append(high_contam)
pot_high_qual=summary[(summary['completeness']>=90) & (summary['contamination']<5)]
contaminated=summary[summary['contamination']>10]

pot_high_qual=pot_high_qual[(pot_high_qual['tRNA']>18) & (pot_high_qual['rRNA']==3)]
print('potential high quality: ',len(pot_high_qual))
print('High completeness but contam >5:', len(high_contam))
print('medium quality', len(med_qual))
print('low_quality', len(low_qual))
print('contaminated', len(contaminated))


potential high quality:  5
High completeness but contam >5: 5
medium quality 35
low_quality 0
contaminated 12


In [19]:
display(Markdown('### Potential High Quality MAGs'))
display(pot_high_qual)

display(Markdown('### High completeness; contamination >5%'))
display(high_contam)

display(Markdown('### Medium quality'))
display(med_qual)

display(Markdown('### Contaminted'))
display(contaminated)

### Potential High Quality MAGs

Bin Id              Marker lineage  completeness  contamination  \
9    bin.49   o__Cytophagales (UID2936)         99.09           0.00   
15   bin.74  p__Bacteroidetes (UID2605)         96.59           2.78   
22  bin.109       k__Bacteria (UID2495)         96.89           1.10   
45  bin.231  p__Bacteroidetes (UID2605)         98.89           2.38   
47  bin.242       k__Bacteria (UID1452)         94.14           2.06   

    heterogeneity contigs   length  tRNA  rRNA  
9             0.0      50  9114085    41     3  
15           25.0     259  4226356    34     3  
22          100.0     115  2397006    44     3  
45            0.0     134  5755058    58     3  
47            0.0      92  2622195    47     3

### High completeness; contamination >5%

Bin Id                 Marker lineage  completeness  contamination  \
0     bin.5  o__Sphingomonadales (UID3310)         95.89           7.03   
14   bin.63   o__Burkholderiales (UID4000)         91.17           7.44   
18   bin.96       o__Rhizobiales (UID3450)         94.62           6.43   
25  bin.122     p__Bacteroidetes (UID2605)         91.53           9.17   
32  bin.165   o__Burkholderiales (UID4002)         92.46           7.51   

    heterogeneity contigs   length  tRNA  rRNA  
0           48.94     118  2907555    46     0  
14          34.62     617  5836948    46     0  
18          25.00     309  3782732    44     1  
25          84.62     553  5868288    47     1  
32          61.54     440  5830594    54     0

### Medium quality

Bin Id                    Marker lineage  completeness  contamination  \
2    bin.19             k__Bacteria (UID2569)         85.28           1.62   
4    bin.22              k__Bacteria (UID203)         67.24           0.00   
6    bin.37             k__Bacteria (UID2495)         79.20           0.95   
7    bin.42          o__Rhizobiales (UID3447)         79.68           9.59   
8    bin.48     o__Sphingomonadales (UID3310)         82.51           4.25   
10   bin.52              k__Bacteria (UID203)         79.29           5.17   
11   bin.53              k__Bacteria (UID203)         70.86           0.00   
12   bin.58             k__Bacteria (UID3187)         79.56           9.62   
17   bin.90      o__Burkholderiales (UID4000)         88.88           3.39   
21   bin.99       f__Micrococcaceae (UID1631)         81.98           3.87   
26  bin.126        p__Bacteroidetes (UID2605)         70.54           2.54   
27  bin.128  c__Deltaproteobacteria (UID3216)         83.08           5.99   
28  bin.142        p__Bacteroidetes (UID2591)         72.19           0.00   
30  bin.152        p__Bacteroidetes (UID2591)         78.91           1.64   
31  bin.158        p__Bacteroidetes (UID2591)         89.88           1.55   
34  bin.174        p__Bacteroidetes (UID2605)         85.88           1.22   
35  bin.175   f__Enterobacteriaceae (UID5054)         80.16           1.34   
36  bin.178        p__Bacteroidetes (UID2591)         88.18           2.22   
37  bin.180             k__Bacteria (UID3187)         68.72           0.00   
40  bin.221             k__Bacteria (UID2495)         77.06           1.69   
41  bin.222     f__Xanthomonadaceae (UID4214)         87.47           4.22   
49  bin.259              k__Bacteria (UID203)         63.07           5.09   
52  bin.273     f__Pseudomonadaceae (UID4490)         67.89           2.63   
53  bin.297  c__Alphaproteobacteria (UID3422)         86.82           5.80   
56  bin.312     o__Sphingomonadales (UID3310)         83.51           4.22   
59  bin.328             k__Bacteria (UID2569)         85.47           0.54   
60  bin.329     f__Xanthomonadaceae (UID4214)         81.44           3.51   
62  bin.336        p__Bacteroidetes (UID2591)         88.29           2.46   
63  bin.339      o__Burkholderiales (UID4105)         87.92           6.88   
65  bin.357              k__Bacteria (UID203)         69.14           0.00   
0     bin.5     o__Sphingomonadales (UID3310)         95.89           7.03   
14   bin.63      o__Burkholderiales (UID4000)         91.17           7.44   
18   bin.96          o__Rhizobiales (UID3450)         94.62           6.43   
25  bin.122        p__Bacteroidetes (UID2605)         91.53           9.17   
32  bin.165      o__Burkholderiales (UID4002)         92.46           7.51   

    heterogeneity contigs   length  tRNA  rRNA  
2           25.00     248  4084233    30     0  
4            0.00     360  3991332    28     0  
6           60.00     228  2620579    29     1  
7           15.79     603  3774911    46     0  
8           51.72     590  4019215    39     1  
10          80.00     434  5538261    44     5  
11           0.00     138  4004477    28     0  
12          23.08     783  5070063    45     0  
17          35.71     456  3738691    35     1  
21          91.67     518  3621491    47     2  
26           0.00     453  2491634    19     1  
27           0.00     917  6998014    47     1  
28           0.00     652  5588805    40     0  
30          20.00    1016  6489973    45     1  
31          12.50     642  4907331    28     1  
34          75.00     427  3242157    22     2  
35          46.67     594  3407361    28     1  
36          14.29     138  4575123    32     1  
37           0.00     568  2614865    17     0  
40         100.00     345  2036184    37     3  
41          28.57     450  3824972    51     2  
49          27.78    1288  6246047    40     0  
52         100.00     409  5994168    42     3  
53          60.61   

### Contaminted

Bin Id                    Marker lineage  completeness  contamination  \
1    bin.11             k__Bacteria (UID2495)         87.91          13.84   
5    bin.24        p__Bacteroidetes (UID2591)         81.38          11.80   
16   bin.84     f__Xanthomonadaceae (UID4214)         95.46          15.09   
20   bin.98  c__Alphaproteobacteria (UID3422)         96.40          10.93   
23  bin.111          o__Rhizobiales (UID3447)         81.19          18.44   
29  bin.151              k__Bacteria (UID203)         92.48          12.76   
33  bin.169        p__Bacteroidetes (UID2605)         95.02          10.17   
42  bin.223   c__Betaproteobacteria (UID3959)         80.21          13.77   
48  bin.252       p__Actinobacteria (UID1454)         91.73          15.43   
54  bin.303      o__Burkholderiales (UID4002)         88.31          13.57   
55  bin.310  c__Alphaproteobacteria (UID3305)         74.40          11.30   
64  bin.356  c__Alphaproteobacteria (UID3422)         84.05          11.31   

    heterogeneity contigs   length  tRNA  rRNA  
1           18.75     270  3189363    35     1  
5           32.35     775  4799165    53     1  
16          15.96     318  4435720    56     0  
20          66.67     190  4462528    50     0  
23          34.07     503  2760438    18     2  
29          42.86     402  2930231    35     0  
33          25.93     422  6455946    56     2  
42          45.45     679  3889267    37     0  
48           7.69     795  4958072    72     3  
54          53.45     414  6176159    59     2  
55          37.50     507  2929400    35     0  
64          44.26     482  3007015    49     0

Let's extract the 16S genes from these for an alternative identification...Their locations are as follows based on the prokka annotations:

| ID | Contig | Location |
|--|--|--|
| Bin.109 | FDLBIOAK_62| 3898..5441 |
| Bin.231 | FBOEMKFD_50 | 71005..71983 |
| Bin.242 | DDBAGHLF_60 | 8878..10393 |
| Bin.49  | LGCKMLPO_5 | 27290..28070 |
| Bin.74  | FMAGHGML_89 | 5179..5862 |

In [ ]:
%%bash
mkdir refinement/16S
extractseq -sformat gb prokka/bin.109/PROKKA_02162021.gbk:FDLBIOAK_62 -regions 3898..5441 -outseq refinement/16S/bin.109.fa
extractseq -sformat gb prokka/bin.231/PROKKA_02162021.gbk:FBOEMKFD_50 -regions 71005..71983 -outseq refinement/16S/bin.231.fa
extractseq -sformat gb prokka/bin.242/PROKKA_02162021.gbk:DDBAGHLF_60 -regions 8878..10393 -outseq refinement/16S/bin.242.fa
extractseq -sformat gb prokka/bin.49/PROKKA_02162021.gbk:LGCKMLPO_5 -regions 27290..28070 -outseq refinement/16S/bin.49.fa
extractseq -sformat gb prokka/bin.74/PROKKA_02162021.gbk:FMAGHGML_89 -regions 5179..5862 -outseq refinement/16S/bin.74.fa

Identification of these 16S sequences was carried out using EZBioCloud to provide a secondary assessment of the sequences origins:

| ID | GTDB classification   | 16S classification        | Best hit                                      | Similarity | Completeness |
|--|--|--|--|--|--|
| Bin.109 | k__Bacteria      | GN04 uncultered candidate | https://www.ncbi.nlm.nih.gov/nuccore/HM186633 | 96.56%     | 95.1%        |
| Bin.231 | p__Bacteroidetes | Sphingobacterium pakistanense | https://www.ncbi.nlm.nih.gov/nuccore/AB610802 | 98.64% | 65.8%        |
| Bin.242 | k__Bacteria      | Fimbriimonas (uncultered) | https://www.ncbi.nlm.nih.gov/nuccore/EF648093 | 91.88% | 100% | 
| Bin.49  | o__Cytophagales  | Sphingobacteriia | https://www.ncbi.nlm.nih.gov/nuccore/AM991273 | 92.47% | 100% |
| Bin.74  | p__Bacteroidetes | Sphingobacteriia | https://www.ncbi.nlm.nih.gov/nuccore/JN391902 | 94.42% | 45.7% | a

Note that many of these classifications have relatively low similarities so will not accurately reflect the correct lineage. 


# Abundance

Since bin.109 is unclassified it won't be included in any abundance assessment ,so let's take a look directly. Just need to take all the outputs of the metahit assembly, label the MAG sequences, align per-sample reads against them and count...
Let's start by labelling the MAG contigs, and recording a list of their ids...

In [27]:
%%bash 
mkdir -p abundance/reference
cd abundance

rm -f reference/*

for mag in $(ls ../magpurify/*.fa); do
    grep '^>' $mag |sed 's/>//' >> reference/mag_contig_ids
    bin=$(basename $mag|sed 's/.fa//')
    cat $mag|sed "s/>/>${bin}_/" >> reference/mags.fa
done

Now take all the assembled contigs (unbinned) produced by metahit, and remove those which form the MAGs...

In [37]:
with open('abundance/reference/mag_contig_ids','r') as fh:
    ids=[line.rstrip('\n') for line in fh]
fh.close()

out_fh=open('abundance/reference/non_mags.fa','w')
with open ('megahit/final.contigs.fa','r') as fh:
    for record in SeqIO.parse(fh,'fasta'):
        if not record.id in ids:
            SeqIO.write([record],out_fh,'fasta')

In order to select the contigs for each MAG from the bam files, we need some intervals to filter alignments with...

In [ ]:
!bin/create_mag_intervals.sh

Now we can separate the per-sample alignments into per-mag alignments, and merge them...

In [49]:
%%bash

qsub bin/extract_all_mag_contigs.sh

In [ ]:
qsub bin/bam_index_and_stat.sh

Now we can use samtool flagstat outputs to get an idea of what contribution the MAGS make to the total proportion of the reads. 


In [34]:
%%bash
readarray -t lines < <(grep -H QC abundance/alignments/*flagstat|awk '{print $1}')
declare -a counts
echo Per-sample counts
echo =================
for line in ${lines[@]}; do
    file=$(echo $line|awk -F: '{print $1}')
    count=$(echo $line|awk -F: '{print $2}')
    file=$(basename $file|awk -F\. '{print $1}')
    counts+=($count)
    echo -e "$file\t$count" >> abundance/per_sample_read_counts.txt
done

total=$(printf "%s\n" ${counts[@]} |awk '{sum+=$1} END {print sum}')
echo Total = $total

# Mags
readarray -t lines < <(grep -H 'mapped (' abundance/per_mag_alignments/*flagstat| sed 's|abundance/per_mag_alignments/||'|grep -v _|awk '{print $1}')
declare -a mag_counts

echo
echo per-MAG counts
echo ==============

for line in ${lines[@]}; do
    file=$(echo $line|awk -F: '{print $1}')
    count=$(echo $line|awk -F: '{print $2}')
    file=$(basename $file|awk -F\. '{print $1"."$2}')
    mag_counts+=($count)
    echo -e "$file\t$count" >> abundance/per_mag_read_counts_full.txt
done
cat abundance/per_mag_read_counts_full.txt|sort -u > abundance/per_mag_read_counts.txt
rm abundance/per_mag_read_counts_full.txt
mags=$(printf "%s\n" ${mag_counts[@]} |awk '{sum+=$1} END {print sum}')
echo Total = $mags

echo -n "Proportion of reads in MAGs: " && echo "$mags $total" |awk '{print $1/$2*100}'

Per-sample counts
Total = 825840492

per-MAG counts
Total = 64278288
Proportion of reads in MAGs: 7.78338


Lets take a look at the number of reads normalised by the length of each MAG to see if there is any relationship between coverage and completeness

In [36]:
mag_read_counts=pd.read_csv('abundance/per_mag_read_counts.txt',sep="\t",header=None,names=('bin','count'))
mag_lengths=summary[["Bin Id","length"]]
mag_read_counts=mag_read_counts.merge(mag_lengths,how='left',left_on='bin',right_on='Bin Id')
mag_read_counts.drop('Bin Id',axis=1,inplace=True)

mag_read_counts['normalised_count']=mag_read_counts['count']/mag_read_counts['length']
mag_read_counts=mag_read_counts.sort_values(by='normalised_count', key=lambda x: np.argsort(index_natsorted(mag_read_counts["normalised_count"])))

display(mag_read_counts)

bin    count   length normalised_count
16  bin.180   165081  2614865         0.063132
5   bin.126   158263  2491634         0.063518
21  bin.221   134907  2036184         0.066255
13  bin.174   241144  3242157         0.074378
17   bin.19   308096  4084233         0.075435
14  bin.175   261674  3407361         0.076797
9   bin.152   512472  6489973         0.078964
10  bin.158   393000  4907331         0.080084
38  bin.310   242940  2929400         0.082932
4   bin.122   496441  5868288         0.084597
49  bin.358   318943  3537498         0.090161
32  bin.259   582499  6246047         0.093259
66   bin.99   338721  3621491         0.093531
25  bin.227   327678  3482264         0.094099
33  bin.268   344083  3629325         0.094806
2   bin.111   284122  2760438         0.102926
12  bin.169   682829  6455946         0.105767
54   bin.52   595528  5538261          0.10753
31  bin.252   555293  4958072         0.111998
6   bin.128   820907  6998014         0.117306
36  bin.297   415853  3473917         0.119707
43  bin.329   261565  2176838         0.120158
62   bin.90   467609  3738691         0.125073
30   bin.24   602574  4799165         0.125558
19  bin.208   433081  3384172         0.127973
55   bin.53   515204  4004477         0.128657
0   bin.109   318858  2397006         0.133023
59   bin.63   821404  5836948         0.140725
60   bin.74   598144  4226356         0.141527
34  bin.272   639139  4418499         0.144651
48  bin.357   566085  3908659         0.144828
58   bin.61   586892  3904703         0.150304
51   bin.42   570278  3774911         0.151071
27  bin.231   922421  5755058          0.16028
40  bin.319   768132  4491870         0.171005
35  bin.273  1058126  5994168         0.176526
42  bin.328   688821  3899706         0.176634
39  bin.312   475511  2590764         0.183541
26   bin.22   792452  3991332         0.198543
47  bin.356   617154  3007015         0.205238
20   bin.21   979557  4771344           0.2053
18  bin.200   852118  4136069         0.206021
63   bin.96   839102  3782732         0.221824
46  bin.339  1151454  5159325         0.223179
11  bin.165  1367124  5830594         0.234474
56    bin.5   709531  2907555          0.24403
15  bin.178  1131659  4575123         0.247351
23  bin.223   999052  3889267         0.256874
24  bin.224  1503350  5793275         0.259499
8   bin.151   827500  2930231         0.282401
64   bin.97   795218  2714296         0.292974
52   bin.48  1182350  4019215         0.294174
3   bin.119  1282807  4233966          0.30298
50   bin.37   813110  2620579         0.310279
53   bin.49  2961083  9114085         0.324891
7   bin.142  2032761  5588805          0.36372
45  bin.336   996919  2669930         0.373388
1    bin.11  1201948  3189363         0.376861
41  bin.324  1710260  4430503         0.386019
57   bin.58  2023983  5070063         0.399203
22  bin.222  1577647  3824972          0.41246
61   bin.84  2082336  4435720         0.469447
29  bin.242  1369661  2622195         0.522334
37  bin.303  3334375  6176159         0.539878
28  bin.238  2890486  4746793         0.608934
44  bin.335  3976015  5152093         0.771728
65   bin.98  3802958  4462528         0.852198

It doesn't look like the raw coverage is a factor in determining wheter we get a high quality MAG or not.

## Enrichment analysis

First, read mapping stats need to be collected for each sample vs each mag from flagstat outputs.

In [37]:
bins=list(summary['Bin Id'].values)

samples=list()
for file in glob.glob('abundance/alignments/*.sorted.dup.bam'):
    sample=(os.path.basename(file).split('.'))[0]
    samples.append(sample)

bin_counts=dict()
paired_re=re.compile('properly paired')
for bin in bins:
    bin_counts[bin]=dict()
    for sample in samples:
        flagstat='abundance/per_mag_alignments/{}_{}.flagstat'.format(bin,sample)
        fh=open(flagstat,'r')
        for line in fh:
            if paired_re.search(line):
                count=(line.split(' '))[0]
                bin_counts[bin][sample]=count
                
counts_df=pd.DataFrame.from_dict(bin_counts,orient='index')            
cols = counts_df.columns.tolist()
cols.sort()
counts_df=counts_df[cols]
counts_df.sort_index(key=lambda x: x.str.replace('bin.','').astype(int),inplace=True)
lengths_df=summary[['Bin Id','length']]
counts_df=lengths_df.merge(counts_df,how='inner',left_on='Bin Id',right_index=True)
counts_df.set_index('Bin Id',drop=True)
counts_df.columns=['Bin Id', 'Length','Elite_1','Elite_2','Elite_3', 'Desert_1','Desert_2','Desert_3',
                   'North_1','North_2','North_3','Bulk_1','Bulk_2','Bulk_3']
counts_df.to_csv('abundance/per_mag_per_bin_counts.txt',sep="\t",index=False)

Enrichment of MAGs between different genotypes were determined using DESeq2 (`mag_enrichment.Rmd`) and plotted:

In [8]:
IFrame('analysis_results/mag_abundance.pdf',width=800,height=800)